In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interactive
from pymongo import MongoClient
from IPython.display import display

# Conectar ao MongoDB
client = MongoClient('mongodb://localhost:27017')
db = client['big_data_ibge']
collection = db['inflacao']

# Buscar os dados do MongoDB
data = pd.DataFrame(list(collection.find()))
data = data.drop(columns=['_id'], errors='ignore') # Remove a coluna _id.

print(data.columns)

# Renomeie as colunas do DataFrame
data.columns = ['Data', 'Ano', 'IPCA']

#Função para gerar gráficos comparativos
def plot_comparacao(filtro):
    # Filtra os dados de acordo com a opção selecionada
    if filtro == 'Últimos 5 anos':
        anos_selecionados = list(range(data['Ano'].max() -5, data['Ano'].max() + 1))
    elif filtro == 'Ano específico':
        ano_especifico = int(input("Digite um ano específico entre (1996-2024): "))
        anos_selecionados = [ano_especifico]
    else:
        anos_selecionados = data['Ano'].unique() # Todos os anos

    # Convertendo para porcentagem para o conjunto filtrado
    df_filtrado = data[data['Ano'].isin(anos_selecionados)]

    if df_filtrado.empty:
        print("Nenhum dado disponível para os anos escolhidos.")
        return

    # Gráfico de Linha para vizualizar a variação do IPCA ao longo do tempo
    plt.figure(figsize=(12, 6))
    sns.lineplot(x='Ano', y='IPCA', hue='Ano', data=df_filtrado, marker='o')
    plt.title('Variação do IPCA ao longo do tempo', fontsize=16)
    plt.ylabel('IPCA (%)', fontsize=14)
    plt.xlabel('Ano', fontsize=14)
    plt.xticks(rotation=45)
    plt.show()

# Widgets para seleção de filtro
filtro_widget = widgets.RadioButtons(
    options=['Todos os anos', 'Últimos 5 anos', 'Ano específico'],
    description='Filtro:',
    style={'description_width': 'initial'}
)

ano_especifico_widget = widgets.IntText(
    value=2020, 
    description='Ano:',
    min=data['Ano'].min(),
    max=data['Ano'].max(),
    style={'description_width': 'initial'}
)

# Função para exibir os widgets adequadamente
def update_plot(filtro, ano_especifico=ano_especifico_widget.value):
    if filtro == 'Ano específico':
        ano_especifico_widget.layout.visibility = 'visible'  # Exibe o campo para o ano específico
    else:
        ano_especifico_widget.layout.visibility = 'hidden'  # Oculta o campo se não for a opção "Ano específico"
    plot_comparacao(filtro, ano_especifico)


# Interatividade
interactive_plot = interactive(plot_comparacao, filtro=filtro_widget)
display(interactive_plot)


Index(['Data', 'Ano', 'IPCA'], dtype='object')


interactive(children=(RadioButtons(description='Filtro:', options=('Todos os anos', 'Últimos 5 anos', 'Ano esp…